### installs

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets scipy ipywidgets matplotlib
!pip install jsonlines
!pip install alpaca-py
!pip install alpaca-trade-api
!pip install websockets==11.0.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

### login

In [ ]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### Training Data

In [ ]:

from datetime import datetime, timedelta
import yfinance as yf
import json
from google.colab import drive


# Fetching the data
ticker = "AAPL"  # Ticker symbol for Dow Jones Industrial Average
# Mount your Google Drive
from google.colab import drive
drive.mount("/content/drive")

# Change directory to where the file is saved
#%cd /content/drive/MyDrive

# Read the data from the file
import pandas as pd
data = yf.download(ticker, start="2023-01-01", end="2023-01-08")  # Example: Data for 2023


import json
import jsonlines

# Step 1: Download the data
# data = yf.download(tickers='GOOG', period='5d', interval='1m')

# Step 2: Convert the data to a list of dictionaries
data_list = data.to_dict('records')
#print(data_list)
# Step 3: Write the data to a .jsonl file
with open('data.jsonl', 'w') as f:
    for item in data_list:
        json.dump(item, f)
        f.write('\n')


"""
Credit for the following code is used from my partner Ethan Zhang that I am using for testing inputting into the model
"""
end_date = datetime.now().date()
start_date = end_date - timedelta(days=7)
# Fetch the DJIA stock data
djia_data = yf.download(ticker, start=start_date, end=end_date)
print(djia_data)

# Convert the DataFrame to a list of dictionaries
djia_records = djia_data.reset_index().to_dict('records')
output_file = "djia_stock_data.jsonl"
with open(output_file, 'w') as f:
    pass

for i in range(7, len(djia_records)):
    input_days = djia_records[i-7:i]
    output_day = djia_records[i]

    # make each input a string and each output a string with the following format:
    # input:
    # "Market Values for previous 6 days:
    # Day 1: Open:{open} High:{high} Low:{low} Close:{close} Adj Close:{adj close} Volume:{volume}, ...
    # Day 6: Open:{open} High:{high} Low:{low} Close:{close} Adj Close:{adj close} Volume:{volume}."
    # output: "Today's Market Values: Open:{open} High:{high} Low:{low} Close:{close} Adj Close:{adj close} Volume:{volume}"
    input_str = "Market Values for previous 6 days:"
    for j in range(6):
        input_str += (
            f"Day {j+1}: Open:{input_days[j]['Open']} High:{input_days[j]['High']} "
            f"Low:{input_days[j]['Low']} Close:{input_days[j]['Close']} Adj "
            f"Close:{input_days[j]['Adj Close']} Volume:{input_days[j]['Volume']}, "
            if j != 5 else
            f"Day {j+1}: Open:{input_days[j]['Open']} High:{input_days[j]['High']} "
            f"Low:{input_days[j]['Low']} Close:{input_days[j]['Close']} Adj "
            f"Close:{input_days[j]['Adj Close']} Volume:{input_days[j]['Volume']}."
        )

    output_str = (
        f"Today's Market Values: Open:{output_day['Open']} High:{output_day['High']} "
        f"Low:{output_day['Low']} Close:{output_day['Close']} Adj "
        f"Close:{output_day['Adj Close']} Volume:{output_day['Volume']}"
    )

    # create a new record with the input and output strings
    record = {"input": input_str, "output": output_str}

    # print(str(record) + "\n")

    # write the record to the JSON Lines file
    with jsonlines.open(output_file, mode='a') as writer:
        writer.write(record)

# Open the .jsonl file
with open(output_file, 'r') as file:
    for line in file:
        json_object = json.loads(line)

        print(json_object)
        print('test')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2024-04-09  168.699997  170.080002  168.350006  169.669998  169.669998   
2024-04-10  168.800003  169.089996  167.110001  167.779999  167.779999   
2024-04-11  168.339996  175.460007  168.160004  175.039993  175.039993   
2024-04-12  174.259995  178.360001  174.210007  176.550003  176.550003   
2024-04-15  175.360001  176.630005  172.500000  172.690002  172.690002   

               Volume  
Date                   
2024-04-09   42451200  
2024-04-10   49709300  
2024-04-11   91070300  
2024-04-12  101593300  
2024-04-15   73531800  


In [ ]:
from datetime import datetime, timedelta
import yfinance as yf
import json
from google.colab import drive


def get_past_6days(ticker):
    # Calculate the date range
    end_date = datetime.now().date()
    start_date = end_date - timedelta(days=6)  # Only need the last 6 days

    # Fetch the stock data for the given range
    data = yf.download(ticker, start=start_date, end=end_date)

    # Reset the index to handle the date column properly
    data.reset_index(inplace=True)

    # Initialize the input string to compile daily values
    input_str = "Market Values for the past 6 days:\n"

    # Loop through each day's data to format it
    for index, row in data.iterrows():
        day_number = index + 1
        daily_values = (
            f"Day {day_number}: Open:{row['Open']} High:{row['High']} "
            f"Low:{row['Low']} Close:{row['Close']} Adj "
            f"Close:{row['Adj Close']} Volume:{row['Volume']}\n"
        )
        input_str += daily_values

    return input_str

ticker = "AAPL"
market_data = get_past_6days(ticker)
print(market_data)



[*********************100%%**********************]  1 of 1 completed

Market Values for the past 6 days:
Day 1: Open:168.8000030517578 High:169.08999633789062 Low:167.11000061035156 Close:167.77999877929688 Adj Close:167.77999877929688 Volume:49709300
Day 2: Open:168.33999633789062 High:175.4600067138672 Low:168.16000366210938 Close:175.0399932861328 Adj Close:175.0399932861328 Volume:91070300
Day 3: Open:174.25999450683594 High:178.36000061035156 Low:174.2100067138672 Close:176.5500030517578 Adj Close:176.5500030517578 Volume:101593300
Day 4: Open:175.36000061035156 High:176.6300048828125 Low:172.5 Close:172.69000244140625 Adj Close:172.69000244140625 Volume:73531800



### Transformer Model

In [ ]:
from transformers import AutoTokenizer
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import matplotlib.pyplot as plt


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "meta-llama/Llama-2-7b-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/FinGPT/llama2-7b-journal-finetune/checkpoint-500")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Model Execution

In [ ]:
#Use model to predict buying or selling the data
eval_prompt = """Given the following financial input data for the past 6 days, Give only one word Buy or Sell:

### Financial Input Data:
""" +  get_past_6days("AAPL") + """
### Output Data:
"""
#print(eval_prompt)
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    output_str = tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=300)[0], skip_special_tokens=True)
    print(output_str)

[*********************100%%**********************]  1 of 1 completed


Given the following financial input data for the past 6 days, Give only one word Buy or Sell:

### Financial Input Data:
Market Values for the past 6 days:
Day 1: Open:168.8000030517578 High:169.08999633789062 Low:167.11000061035156 Close:167.77999877929688 Adj Close:167.77999877929688 Volume:49709300
Day 2: Open:168.33999633789062 High:175.4600067138672 Low:168.16000366210938 Close:175.0399932861328 Adj Close:175.0399932861328 Volume:91070300
Day 3: Open:174.25999450683594 High:178.36000061035156 Low:174.2100067138672 Close:176.5500030517578 Adj Close:176.5500030517578 Volume:101593300
Day 4: Open:175.36000061035156 High:176.6300048828125 Low:172.5 Close:172.69000244140625 Adj Close:172.69000244140625 Volume:73531800

### Output Data:
Day 5: Open:173.099998046875 High:173.160003125 Low:169.5600009765625 Close:170.140006103515625 Adj Close:170.140006103515625 Volume:28512400
Day 6: Open:170.429998046875 High:170.800003125 Low:168.39000390625 Close:169.0699990234375 Adj Close:169.069999

In [ ]:
#Determine decision based on Model Output
split_str = output_str.split("Output Data:")
print("Buy" in split_str[1])
decision = 'B' if "Buy" in split_str[1] else 'S'
print(decision)

False
S


### Alpaca

In [ ]:
%cd /content/drive/MyDrive/trading_bot/

/content/drive/MyDrive/trading_bot


In [ ]:
# new_eval = "Respond with one word, Buy or Sell if this data indicates buying or selling this stock" + output_str
# model_input = tokenizer(new_eval, return_tensors="pt").to("cuda")
# ft_model.eval()
# with torch.no_grad():
#     temp = tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=300)[0], skip_special_tokens=True)
#     print(temp)
# hello


Respond with one word, Buy or Sell if this data indicates buying or selling this stock Given the following financial input data for the past 4 days, Respond with one word, Buy or Sell

### Financial Input Data:
Market Values for previous 4 days:
Day 1: Open:171.190002 High:171.250000 Low:169.479996 Close:170.029999 Adj Close:170.029999 Volume:46240500,
Day 2: Open:169.080002 High:169.339996 Low:168.229996 Close:168.839996 Adj Close:168.839996 Volume:49329500,
Day 3: Open:168.789993 High:170.679993 Low:168.580002 Close:169.649994 Adj Close:169.649994 Volume:47602100,
Day 4: Open:170.289993 High:171.919998 Low:168.820007 Close:168.820007 Adj Close:168.820007 Volume:53704400,

### Output Data:
Day 5: Open:168.789993 High:168.789993 Low:166.809994 Close:166.809994 Adj Close:166.809994 Volume:31035500,
Day 6: Open:167.729994 High:168.580002 Low:166.820007 Close:167.729994 Adj Close:167.729994 Volume:33544100,
Day 7: Open:168.039996 High:169.280004 Low:168.039996 Close:168.990004 Adj Close:1

In [ ]:
from trade import *

api = setup()

symbol = input('Enter a Stock Ticker to trade ==> ')
info = get_past_6days(symbol)
print(info)
#decision = input('Enter B/S for buying and selling ==> ')
qty = input('Enter the number of stocks you wish to buy/sell ==> ')

trade_stock(api, symbol, decision, qty)

Enter a Stock Ticker to trade ==> AAPL


[*********************100%%**********************]  1 of 1 completed


Market Values for the past 6 days:
Day 1: Open:168.8000030517578 High:169.08999633789062 Low:167.11000061035156 Close:167.77999877929688 Adj Close:167.77999877929688 Volume:49709300
Day 2: Open:168.33999633789062 High:175.4600067138672 Low:168.16000366210938 Close:175.0399932861328 Adj Close:175.0399932861328 Volume:91070300
Day 3: Open:174.25999450683594 High:178.36000061035156 Low:174.2100067138672 Close:176.5500030517578 Adj Close:176.5500030517578 Volume:101593300
Day 4: Open:175.36000061035156 High:176.6300048828125 Low:172.5 Close:172.69000244140625 Adj Close:172.69000244140625 Volume:73531800

Enter the number of stocks you wish to buy/sell ==> 10
Placing a sell order for 10 shares of AAPL at $169.34
